## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN','NAME'], axis=1, inplace=True)


In [3]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
all_types = application_df['APPLICATION_TYPE'].value_counts()
all_types

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

#updated the cutoff number from 200 to 1000 to make consolidated more of the lesser-used categories
cutoff = 1000
application_types_to_replace = []

for type, count in all_types.items():
    if count < cutoff:
        application_types_to_replace.append(type)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
all_class = application_df['CLASSIFICATION'].value_counts()
all_class

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
threshold = 1
filtered = all_class[all_class > threshold]
filtered

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

#Again, updated the cutoff here to 2000 from 1750
cutoff = 2000
classifications_to_replace = []

for item, count in all_class.items():
    if count < cutoff:
        classifications_to_replace.append(item)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: count, dtype: int64

In [10]:
#Checking out the range of ask amounts for each campaign
all_asks = application_df['ASK_AMT'].value_counts()
all_asks

ASK_AMT
5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: count, Length: 8747, dtype: int64

In [11]:
# I notice a lot of campaigns with 5000 as the ask amount. I'm going to set this as the only value and the rest to other and see what the model shows
# use the variable name `amts_to_replace`

cutoff = 5000
# Get all unique values in the column
amts_to_replace = application_df['ASK_AMT'].unique()

# Keep values that are not equal to cutoff
amts_to_replace = [amt for amt in amts_to_replace if amt != cutoff]  

# Display values to be replaced for debugging
#print("Amounts to replace:", amts_to_replace)

# Replace values not equal to cutoff with 'Other'
application_df['ASK_AMT'] = application_df['ASK_AMT'].apply(lambda x: 'Other' if x != cutoff else x)

# Check to make sure replacement was successful
print(application_df['ASK_AMT'].value_counts())

ASK_AMT
5000     25398
Other     8901
Name: count, dtype: int64


In [12]:
#creating a copy of the original dataframe
application_df_copy = application_df.copy()

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_copy = pd.get_dummies(application_df_copy)
application_df_copy.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_5000,ASK_AMT_Other
0,1,1,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
1,1,1,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
2,1,0,False,False,False,False,True,False,True,False,...,False,False,False,False,False,False,True,False,True,False
3,1,1,False,False,True,False,False,False,True,False,...,False,False,False,False,False,False,True,False,False,True
4,1,1,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,True


In [14]:
# Split our preprocessed data into our features and target arrays
y = application_df_copy['IS_SUCCESSFUL'].values
X = application_df_copy.drop(['IS_SUCCESSFUL'], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 10
#adding an extra node to see if the model performs better
hidden_nodes_layer3 =  10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn.summary()

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │           400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 631 (2.46 KB)

 Trainable params: 631 (2.46 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
#added an extra 100 epochs
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 306us/step - accuracy: 0.6480 - loss: 0.6410
Epoch 2/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 250us/step - accuracy: 0.7189 - loss: 0.5703
Epoch 3/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 249us/step - accuracy: 0.7299 - loss: 0.5615
Epoch 4/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 246us/step - accuracy: 0.7268 - loss: 0.5644
Epoch 5/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 245us/step - accuracy: 0.7309 - loss: 0.5597
Epoch 6/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 246us/step - accuracy: 0.7316 - loss: 0.5573
Epoch 7/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 246us/step - accuracy: 0.7333 - loss: 0.5536
Epoch 8/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 245us/step - accuracy: 0.7318 - loss: 0.5556
Epoch 9/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 247us/step - accuracy: 0.7234 - loss: 0.5620
Epoch 10/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 246us/step - accuracy: 0.7330 - loss: 0.5536
Epoch 11/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 245us/step - accuracy: 0.7338 - loss: 0.5532
Epoch 12/200
804/80

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 472us/step - accuracy: 0.7227 - loss: 0.5621
Loss: 0.562097430229187, Accuracy: 0.7226822376251221


In [20]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_optimization.h5')